In [11]:
import pandas as pd
import numpy as np

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

output_width = 1000
#output_width = 80 #//*** Normal Output width

#//*** Normal Output width
pd.set_option("display.width", output_width)

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', None)
#pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)


Work with the Data for Term Project

Data is downloaded using DSC540 StoneburnerKurt TermProject _ Load_PreProcess_Data
which I supposed should be added as an external library if this was a production project.

Coded is separated to keep the project more manageable

In [12]:
#//*************************************
#//*** Import stored/preprocessed data
#//*************************************

#//*** Columns to remove from imported CSVs. We should be able to kill these on import if we were cool.
#//*** But we're not, so we'll use an expedient column delete list.

del_cols = ['Unnamed: 0', '_id']
#//*** Load datframes from file, because we mess them up
covid_ethnic_df = pd.read_csv("z_covid_ethnic_df.csv")
covid_cases_df = pd.read_csv("z_covid_cases_df.csv")
pop_attrib_df = pd.read_csv("z_pop_attrib_df.csv")
covid_project_df = pd.read_csv("z_covid_project_df.csv")

#//***********************************************************************************
#//*** Remove excess columns from read_csv
#//*** Use the loop in case we need to delete columns that are not exclusive to all
#//***********************************************************************************
for x in del_cols:
    if x in covid_cases_df.columns:
        covid_cases_df.drop([x], axis=1, inplace=True)

    if x in covid_ethnic_df.columns:
        covid_ethnic_df.drop([x], axis=1, inplace=True)
    
    if x in pop_attrib_df.columns:
        pop_attrib_df.drop([x], axis=1, inplace=True)
    
    if x in covid_project_df.columns:
        covid_project_df.drop([x], axis=1, inplace=True)
        
#print(covid_cases_df.head())
#print(covid_ethnic_df.head())
#print(pop_attrib_df.head())
#print(covid_project_df.head())



In [13]:
#//*** Combine Pacific Islander and Hawaiian values into one.
#//*** Hawaiian isn't a good choice of variable name. It's kind of racist and definitely non-inclusive.
#//*** But I need this to work, before I can reconsider a different variable name

covid_ethnic_df['race_ethnicity']=covid_ethnic_df['race_ethnicity'].str.replace('Native Hawaiian or Pacific Islander','Hawaiian')
covid_ethnic_df['race_ethnicity']=covid_ethnic_df['race_ethnicity'].str.replace('Native Hawaiian and other Pacific Islander','Hawaiian')
covid_ethnic_df['race_ethnicity']=covid_ethnic_df['race_ethnicity'].str.replace('Multi-Race','Multiracial' )

In [14]:
#print(pop_attrib_df)

HTML(pop_attrib_df.to_html())

,cty_fibs,county,population,Latino,White,Asian,Black,American Indian or Alaska Native,Hawaiian,Other,0rf_num,0rf_rate,0rf_err,1-2rf_num,1-2rf_rate,1-2rf_err,3plrf_num,3plrf_rate,3plrf_err
0,1,Alameda,1671329,373055,512134,529698,169954,4157,13474,68857,52135,3.16,1.99,1159312,70.37,6.98,436060,26.47,6.84
1,3,Alpine,1129,139,692,18,4,243,0,33,241,21.89,8.40,579,52.59,10.35,281,25.52,9.31
2,5,Amador,39752,5753,30742,575,994,606,83,999,7169,20.35,5.97,17817,50.59,7.16,10235,29.06,6.49
3,7,Butte,219186,37731,155415,10573,3526,3390,465,8086,44433,19.46,5.32,122804,53.79,6.88,61078,26.75,6.02
4,9,Calaveras,45905,5967,36672,719,420,562,98,1467,9940,21.93,6.38,22900,50.53,7.46,12478,27.53,6.60
5,11,Colusa,21547,13018,7344,266,220,293,73,333,4223,19.65,5.75,12343,57.45,7.10,4920,22.90,6.19
6,13,Contra Costa,1153526,300420,492393,204045,100798,3126,5379,47365,150886,13.16,3.91,710710,61.99,6.99,284932,24.85,6.49
7,15,Del Norte,27812,5596,17236,802,917,2059,42,1160,5189,20.77,6.47,13874,55.54,7.66,5915,23.68,6.47
8,17,El Dorado,192843,25378,148903,8974,1696,1500,328,6064,55253,29.06,6.34,94483,49.69,7.03,40421,21.26,5.76
9,19,Fresno,999101,537180,286049,103430,46274,5967,1437,18764,68407,6.95,3.13,612805,62.24,7.10,303388,30.81,6.79


In [15]:
#//*** Sort Time Series by date and reset index
covid_cases_df = covid_cases_df.sort_values(by='date')
covid_ethnic_df = covid_ethnic_df.sort_values(by='date')

#//*** Reset the index
#covid_ethnic_df.reset_index(inplace=True)





#//*** Get first Ethnic_df date
ethnic_start_date = covid_ethnic_df['date'].iloc[0]
print(f"Ethinic State: {ethnic_start_date}")

#//*************************************************************************************************
#//*** Get the iloc (index #) of the first covid_case_df entry to match the date in covid_ethic_df
#//*** Compound code
#//*** 1. Get the entries where the date matches ethnic start date
#//*** 2. Get the first value from the list
#//*** 3. Get the Index (name) of that entry
#//*** 4. Get the iloc value of the name entry. This is the value to slice from covid_cases_df
#//*************************************************************************************************
#//*** I hate these, but I see the appeal
#//*************************************************************************************************
covid_start_iloc = covid_cases_df.index.get_loc(covid_cases_df[ covid_cases_df['date'] == ethnic_start_date].iloc[0].name)

#print(covid_cases_df.iloc[covid_start_iloc])
#//*** Merge Time Series covid_ethnic_df - covid_cases_df
covid_cases_df = covid_cases_df.iloc[covid_start_iloc:]

#//*** Start the Bg Table DF with a subset of
#bt_df = covid_ethnic_df
#print
#print(covid_ethnic_df.head())
#print(bt_df.head())
    
    
bt_df = pd.merge(covid_ethnic_df,covid_cases_df,how='left', on='date')

bt_df = pd.merge(bt_df,pop_attrib_df,how='left', on='county')

print(len(bt_df))

for group in bt_df.groupby('date'):
    print(group[1])
    break

Ethinic State: 2020-04-13T00:00:00
140640
    race_ethnicity  cases  case_percentage  deaths  death_percentage  percent_ca_population                 date        county  totalcountconfirmed  totalcountdeaths  newcountconfirmed  newcountdeaths  cty_fibs  population     Latino     White     Asian     Black  American Indian or Alaska Native  Hawaiian    Other   0rf_num  0rf_rate  0rf_err  1-2rf_num  1-2rf_rate  1-2rf_err  3plrf_num  3plrf_rate  3plrf_err
0           Latino   5276            35.99     170             28.38                   38.9  2020-04-13T00:00:00  Contra Costa                581.0              13.0                 28               1      13.0   1153526.0   300420.0  492393.0  204045.0  100798.0                            3126.0    5379.0  47365.0  150886.0     13.16     3.91   710710.0       61.99       6.99   284932.0       24.85       6.49
1           Latino   5276            35.99     170             28.38                   38.9  2020-04-13T00:00:00     El Dorado    

In [20]:
#//*** Rename Columns
g = {
    "bt_rename" : {
        "race_ethnicity" : "race",
        "case_percentage" : "case_percent",
        "death_percentage" : "death_percent",
        "percent_ca_population" : "percent_ca_pop",
        "totalcountconfirmed" : "confirmed",
        "totalcountdeaths" : "deaths",
        "newcountconfirmed" : "newconfirm",
        "newcountdeaths" : "newdeath",
        "population" : "pop",
        "American Indian or Alaska Native" : "native"
    }
}

cols = list(hier_df.columns)

for x in range(0,len(cols)):
    print(x)

hier_df = bt_df.set_index([bt_df['date'],bt_df['county']])
print(hier_df)

#HTML(hier_df.to_html())

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
                                 race_ethnicity   cases  case_percentage  deaths  death_percentage  percent_ca_population                 date        county  totalcountconfirmed  totalcountdeaths  newcountconfirmed  newcountdeaths  cty_fibs  population     Latino     White     Asian     Black  American Indian or Alaska Native  Hawaiian    Other   0rf_num  0rf_rate  0rf_err  1-2rf_num  1-2rf_rate  1-2rf_err  3plrf_num  3plrf_rate  3plrf_err
date                county                                                                                                                                                                                                                                                                                                                                                                                                                                 
2020-04-13T00:00:00 Contra Costa

In [17]:
#//*** Let's run some correllations for funsies
pop_cols = ['population', 'Latino','White','Asian','Black','American Indian or Alaska Native', 'Hawaiian', 'Other' ]
rf_cols = ['0rf_num', '1-2rf_num',  '3plrf_num']
rf_cols = ['0rf_rate', '1-2rf_rate',  '3plrf_rate']

for x in pop_cols:
    for y in rf_cols:
        print(f"{x} {y} - {pop_attrib_df[ [x,y] ].corr().iloc[0].iloc[1] }")

population 0rf_rate - -0.4567961826407012
population 1-2rf_rate - 0.45180422307614304
population 3plrf_rate - 0.1425255103467081
Latino 0rf_rate - -0.40905527183588614
Latino 1-2rf_rate - 0.3837575774517347
Latino 3plrf_rate - 0.1806483757865183
White 0rf_rate - -0.464199120421134
White 1-2rf_rate - 0.4797158976341499
White 3plrf_rate - 0.09234516802071475
Asian 0rf_rate - -0.5087228664495507
Asian 1-2rf_rate - 0.5231641998293637
Asian 3plrf_rate - 0.1079399281808924
Black 0rf_rate - -0.38279510407369116
Black 1-2rf_rate - 0.3729504695343319
Black 3plrf_rate - 0.13387953801240074
American Indian or Alaska Native 0rf_rate - -0.3905801458839876
American Indian or Alaska Native 1-2rf_rate - 0.3961581308641285
American Indian or Alaska Native 3plrf_rate - 0.09669106495836434
Hawaiian 0rf_rate - -0.5532497192513318
Hawaiian 1-2rf_rate - 0.5874851204300123
Hawaiian 3plrf_rate - 0.0700536245231904
Other 0rf_rate - -0.5086191874382465
Other 1-2rf_rate - 0.5291907548607251
Other 3plrf_rate - 0.